# Reinforcement learning

## Description of the problem

An **entity** can be is a **state** $s$. Each state has associated a **reward** $R(s)$. From that state it can take **actions** $a$ that will take the entity to another state $s'$. The state, its reward, an action that can be taken in that state, and the state the entity will get to with that action can be represented with a tuple:

$$(s, a, R(s), s')$$

**Return** is the sum of the rewards from a sequence of states and actions, wieghted by a discount factor that compounds along the series of actions. 

$$return = R(s) + \gamma R(s') + \gamma^2 R(s'') + \dots$$

A **Policy** is a function $\Pi$ that for a given state gives us the action $a$ to take next.

The objective of **reinforcement learning algorithm** is to find a policy $\Pi(s) = a$ that maximizes the return.

This problem formulation is a **Markov Decision Process (MDP)**. In a MDP, the future only depends on the current state, regardless of how we have gotten to that state.

## State action value function

We define the **state action value function** (sometimes also called *Q-function*) $Q(s, a)$ is the return if you,
* start in state $s$
* take action $a$
* behave optimally after that

The best possible return from state s is $\max_{a} Q(s,a)$

## Bellman equation

$$Q(s, a) = R(s) + \gamma \max_{a'} Q(s', a')$$

Where $a'$ is each of the actions that can be taken in state $s'$

## Random (stochastic) environment

In this case the return of a sequence of states is

return = $$E[R(s) + \gamma R(s') + \gamma^2 R(s'') + \dots ]$$

And the Belmman equation is:

$$Q(s,a) = R(s) + \gamma E[\max_{a'} Q(s', a')]$$

## Continuous state spaces

Example: state space for a car, in a 2-D world,

$$\begin{bmatrix}x \\ y \\ \theta \\ \dot{x} \\ \dot{y} \\ \dot{\theta}\end{bmatrix}$$

where $x$ and $y$ are the coordinates, $\theta$ is the orientation (angle), and $\dot{x}$, $\dot{y}$, $\dot{\theta}$ the rate of change along each coordinate and the orientation.

## Learning the state action value function

Idea: Train a neural network to calculate, for a given state $s$, the return of the state action value functions for the actions possible in that state, so we can choose the one with largest $Q(s, a)$. In other words, train a neural network that given s and a returns $y \approx Q(a,a)$. Or, in less words, train the neural network to learn the Bellman equation.

To do so, we can create a large set of tuples

$$(s^{(1)}, a^{(1)}, R(s^{(1)}), s'^{(1)}) \\ (s^{(2)}, a^{(2)}, R(s^{(2)}), s'^{(2)}) \\ \dots$$

And then, the training examples for the neural network will be:

* For the inputs $x$, each of the tuples 
$$(s^{(1)}, a^{(1)}), (s^{(2)}, a^{(2)}), \dots$$
* For the target values y, the corresponding 
$$Q(s^{(1)},a^{(1)}), Q(s^{(2)},a^{(2)}), \dots$$

calculated with the Bellman equation, for example

$$Q(s^{(1)}, a^{(1)}) = R(s^{(1)}) + \gamma \max_{a'} Q(s'^{(1)}, a')$$

Note that the target values $y$ depend only on the last two elements of the tuples $(s^{(i)}, a^{(i)}, R(s^{(i)}), s'^{(i)})$

At the begining, we don't know the $Q(s, a)$ function, but it can be initialized randomly. In every step, it will get better.

Learning algorithm (sometimes call the **Deep-Q network**)

<pre>
    Initialize Q_nn neural network randomly as guess of Q(s, a)
    Repeat {
        Take actions to generate tuples (s, a, R(s), s')
        Store the 10,000 more recent examples of these tuples (replay buffer)
        Train neural network:
            Create training set of 10,000 examples x, y using
                x = (s, a) and y = R(s) + &gamma; max<sub>a'</sub> Q_nn(s', a')
            Train Q<sub>new</sub> such that Q<sub>new</sub>(s, a) &asymp; y
        Set Q_nn = Q<sub>new</sub>
    }
</pre>

> Note: It is not clear in the lecture, I interpret "take actions to generate tuples" in the lesson as to take sequence of actions until you reach a final state. Refer to the ideas in the "Search" chapter of the *CS50: AI with Python* course in edX. Maybe not, since here we are just trying to generate training samples to calculate $Q(s, a)$

One possible architecture of the neural network is (from course example for lunar lander, with 8 parameters for the state,and 4 possible actions, one hot encoded):

```pyhton
tf.keras.models.Sequential ([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
]) 
```

An improved architecture uses (for this case) four units in the output layer, to compute at the same time the $Q(s, a)$ function for all the possible actions in one state. The input, in this case, is the 8 parameters that represent the state.

```pyhton
tf.keras.models.Sequential ([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4)
]) 
```

> Note: With this network configuration, the standard way to calculate the loss won't work,as the Bellman equation uses the four outputs. See how it is done in the implementation example bellow.

## Algorithm refinement: $\epsilon$-greedy policy

How to improve the "`Take actions to generate tuples (s, a, R(s), a')`" step in the algorithm?

> Note to self: again, refer to the "Search" chapter of the *CS50: AI with Python* course in edX.

Instead of taking the actions randomly, use the following algorithm,

<pre>
    With probability (1- &epsilon;) pick the action a that maximizes Q(s, a)
    With probability &epsilon; pick an action a randomly
</pre>

## Algorithm refinement: mini-batch and soft updates

### Mini-bacthes

This refinement also applies to linear regression or the training of a neural network.

Idea: instead of using all the samples to calculate the cost function in each step of the gradient decent algorithm, do it using a subset (*batch*) of the trainign examples (e.g., with a training set of 1,000,000, use a batch of 1,000 examples).

### Soft updates

The last step in the algorithm was to replace the $Q(s, a)$ function with the newly calculated $Q_{new}(s, a)$. Doing so, it can create abrupt changes in the Q function, sometings replacing a somehow good function by a worse one.

The idea is instead of replacing the parameters of the newural network with the new ones, replace them so that:

$$
   W = 0.01 W_{new} + 0.99 W \\
   B = 0.01 B_{new} + 0.99 B
$$

In [ ]:
import random
from collections import deque, namedtuple

import numpy as np
import matplotlib.pyplot as plt

import gym
import tensorflow as tf
from tensorflow import keras


In [ ]:
STATE_FEATURES = 1
STATES_N = 64
ACTIONS = [0, 1, 2, 3]
EPSILON = 0.01
GAMMA = 0.98
ALPHA = 0.001
TAU = 0.001

#ITERATIONS = 10000
#UPDATE_STEP = 500
#TRAIN_SAMPLES = 100
ITERATIONS = 500
UPDATE_STEP = 20
TRAIN_SAMPLES = 20

DEBUG = True
RENDER = False

In [ ]:
def create_NN():
    """
    Defines and compiles the neural network
    """
    nnetwork = tf.keras.models.Sequential ([
        tf.keras.layers.Input(shape=STATE_FEATURES),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(len(ACTIONS))
    ])

    nnetwork.compile(
        optimizer=keras.optimizers.Adam(learning_rate=ALPHA),
        loss=tf.keras.losses.MeanSquaredError()
    )

    return nnetwork

In [ ]:
def egreedy_select(state, qnetwork, epsilon):
    """
    Given a state and a state action value function (implemented with a q-network) selects an
    action implementing the e-greedy policy

    Arguments:
        state:
        qnetwork:
        epsilon:

    Returns:
        action:
    """
    
    if np.random.choice([True, False], p=[1 - epsilon, epsilon]):
        # Here the greedy selection of next action
        # Transform dimensions of state for qnetwork
        state_qn = np.expand_dims(state, axis=0)
        q_values = qnetwork(state_qn).numpy()
        if np.unique(q_values).shape[0] == 1:
            # All q_values are the same, we do a random select
            action = np.random.choice(ACTIONS)
        else:   
            # Selection of the action with max q_value
            action = q_values.argmax(axis=1)
    else:
        action = np.random.choice(ACTIONS)
    
    return int(action)
    

In [ ]:
def compute_loss(samples, q_new, q_nn, gamma):

    states = tf.convert_to_tensor(
        np.array([s.state for s in samples if s is not None]), dtype=tf.float32
    )
    actions = tf.convert_to_tensor(
        np.array([s.action for s in samples if s is not None]), dtype=tf.float32
    )
    rewards = tf.convert_to_tensor(
        np.array([s.reward for s in samples if s is not None]), dtype=tf.float32
    )
    states_prime = tf.convert_to_tensor(
        np.array([s.state_prime for s in samples if s is not None]), dtype=tf.float32
    )
    done_values = tf.convert_to_tensor(
        np.array([s.done for s in samples if s is not None]), dtype=tf.float32
    )

    # Bellman equation
    qvalues_target = rewards + gamma * (1 - done_values) * tf.reduce_max(q_nn(states_prime), axis=-1)

    # Get the q_values
    q_values = q_new(states)
    q_values = tf.gather_nd(q_values, tf.stack([tf.range(q_values.shape[0]),
                                                tf.cast(actions, tf.int32)], axis=1))
        
    return tf.keras.losses.MSE(qvalues_target, q_values)

In [ ]:

@tf.function
def train_nn(samples, gamma, optimizer, q_new, q_nn):
    """
    Updates the weights of the Q networks.
    
    Args:
      experiences: (tuple) tuple of ["state", "action", "reward", "next_state", "done"] namedtuples
      gamma: (float) The discount factor.
    
    """
    
    with tf.GradientTape() as tape:
        loss =  compute_loss(samples, q_new, q_nn, gamma)

    # Get the gradients of the loss with respect to the weights.
    gradients = tape.gradient(loss, q_new.trainable_variables)
    
    # Update the weights of the q_network.
    optimizer.apply_gradients(zip(gradients, q_new.trainable_variables))

    return

In [ ]:
q_nn = create_NN()
q_new = create_NN()

optimizer=keras.optimizers.Adam(learning_rate=ALPHA)

In [ ]:

if RENDER:
    env = gym.make('FrozenLake-v1', desc=None, map_name="8x8", is_slippery=True, render_mode="human")
else:
    env = gym.make('FrozenLake-v1', desc=None, map_name="8x8", is_slippery=True)
state, _ = env.reset()

In [ ]:
# buffer to store the tupples (state, action, reward, state_prime)
buffer = deque(maxlen=UPDATE_STEP)

# Namedtuple to hold the data points
data_point = namedtuple("data_point", ["state", "action", "reward", "state_prime", "done"])

if DEBUG:
    # Structure to keep track of visited states and plot a heatmap:
    #   Rows: update iteration
    #   Columns: states
    #   Values: number of visits to each sate
    heatmaps = np.zeros((UPDATE_STEP, STATES_N))

In [ ]:
for i in range(ITERATIONS):

    action = egreedy_select(state, q_nn, EPSILON)
    state_prime, reward, done, _, _ = env.step(action)

    if RENDER:
        env.render()
    if DEBUG:
        heatmaps[(i+1) % UPDATE_STEP, state] += 1

    buffer.append(data_point(state, action, reward, state_prime, done))
    if done:
        # Go back to initial state
        state, _ = env.reset()
    else:
        state = state_prime

    if ((i+1) % UPDATE_STEP) == 0:
        samples = random.sample(buffer,k=TRAIN_SAMPLES)
        
        # Train the neural network
        train_nn(samples, GAMMA, optimizer, q_new, q_nn)

        # update the weights of target q_network
        for q_nn_weights, q_new_weights in zip(
            q_nn.weights, q_new.weights
        ):
            q_nn_weights.assign(TAU * q_new_weights + (1.0 - TAU) * q_nn_weights)

In [ ]:

if DEBUG:
    n_heatmaps = heatmaps.shape[0]
    ncols = 4
    fig, axs = plt.subplots(
        nrows= n_heatmaps // ncols, 
        ncols=ncols, 
        figsize=[18,18]
    )
    for i in range(n_heatmaps):
        axs[i // ncols , i % ncols].imshow(np.reshape(heatmaps[i,:], (8,8)), cmap="Greens")


In [ ]:
if RENDER:
    env.close()


In [ ]:

# Unit tests

import unittest 

class TestNotebook(unittest.TestCase):

    def test_egreedy_select1(self):
        '''
        Test selection of the state with highest state action value
        '''
        nn = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=STATE_FEATURES),
            tf.keras.layers.Dense(len(ACTIONS))
        ])

        nn.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.01),
            loss=tf.keras.losses.MeanSquaredError()
        )

        # State is irrelevant here
        state = 5

        # Set weights of nn so action 2 has the highest state action value 
        weights = np.array([np.array([[0., 1., 2., 1.]]), np.array([0., 0., 0. ,0.])], dtype='object')
        nn.set_weights(weights)
        
        self.assertEqual(egreedy_select(state, nn, epsilon=0), 2)

    def test_egreedy_select2(self):
        '''
        Test random selection of the state when all state action values are the same
        '''
        nn = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=STATE_FEATURES),
            tf.keras.layers.Dense(len(ACTIONS))
        ])

        nn.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.01),
            loss=tf.keras.losses.MeanSquaredError()
        )

        # State is irrelevant here
        state = 5

        # Set weights 
        weights = np.array([np.array([[1., 1., 1., 1.]]), np.array([0., 0., 0. ,0.])], dtype='object')
        nn.set_weights(weights)
        
        np.random.seed(66)
        self.assertEqual(egreedy_select(state, nn, epsilon=0), 3)

    def test_egreedy_select3(self):
        '''
        Test random selection with epsilon <> 0
        '''
        nn = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=STATE_FEATURES),
            tf.keras.layers.Dense(len(ACTIONS))
        ])

        nn.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.01),
            loss=tf.keras.losses.MeanSquaredError()
        )

        # State is irrelevant here
        state = 5

        # Set weights 
        weights = np.array([np.array([[0., 1., 2., 3.]]), np.array([0., 0., 0. ,0.])], dtype='object')
        nn.set_weights(weights)
        
        np.random.seed(66)
        self.assertEqual(egreedy_select(state, nn, epsilon=1), 3)

unittest.main(argv=[''], verbosity=2, exit=False)